# Creacion de datos en sqlite3

## Importar librerias

In [1]:
import sqlite3
import pandas as pd
from faker import Faker
import random

## Creacion de lista con datos personales para 50000 estudiantes

In [2]:
fk = Faker('es_ES')
data = []
domain = ['yahoo.com','hotmail.com','gmail.com', 'outlook.com','soho.com']
for x in range(1,500001):
    first_name = fk.first_name()
    last_name = fk.last_name()
    name = first_name + " "+last_name
    email = first_name[0].lower()+last_name.lower()+"@"+domain[random.randint(0,len(domain)-1)]
    data.append((x,name,email))

## Creacion de la base de datos "students.db"

In [3]:
conn = sqlite3.connect('students.db')
cursor = conn.cursor()
# Crear la tabla 'personal_data'
cursor.execute("""
CREATE TABLE IF NOT EXISTS personal_data (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nombre TEXT NOT NULL,
    correo TEXT NOT NULL);
""")

## Creacion de la tabla "personal_data"

In [4]:
cursor.executemany("""
INSERT INTO personal_data (id, nombre, correo)
VALUES (?, ?, ?);
""", data)

# Confirmar los cambios y cerrar la conexión
conn.commit()
conn.close()



## Visualizacion de datos

In [5]:
conn = sqlite3.connect('students.db')
df = pd.read_sql_query('SELECT * from personal_data', conn)
conn.close()
df.head()

,id,nombre,correo
0,1,Julia Lara,jlara@yahoo.com
1,2,Jennifer Garcés,jgarcés@yahoo.com
2,3,Melisa Gimenez,mgimenez@outlook.com
3,4,Carmela Gomis,cgomis@yahoo.com
4,5,Venceslás Borrell,vborrell@yahoo.com
